# 1. Set up

In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# import re
# import json
# import gensim.downloader as api

# !pip install langid
# import langid

# !pip install gensim
# from gensim.models import Word2Vec

# !pip install -U spacy
# !python -m spacy download it_core_news_sm
# import spacy

# !pip install sense2vec
# from sense2vec import Sense2Vec

# import random

# Find Distractores


# 1. Attempt Sense2Vec
- Problems: 

            1. Not a italian model so it works only for english words
            2. It's not clear how to use the trained model, lack of documentation(at least that I have found :))

- Solutions to the problem:

            1. Use an Italian sense2vec model
            2. translate from italian to english and then back (canc ause miss translation)

1. Sense2Vec (https://jovian.com/sunilbabu1981/qg-colab-1-01-generating-distractors-using-sense2vec)

In [2]:
from sense2vec import Sense2Vec

# Download a pre-trained Sense2Vec model
!tar -xvf s2v_reddit_2015_md.tar.gz

# Load the Sense2Vec model
s2v_model_path = "s2v_old"
s2v = Sense2Vec().from_disk(s2v_model_path)

x ./._s2v_old
x ./s2v_old/
x ./s2v_old/._freqs.json
x ./s2v_old/freqs.json
x ./s2v_old/._vectors
x ./s2v_old/vectors
x ./s2v_old/._cfg
x ./s2v_old/cfg
x ./s2v_old/._strings.json
x ./s2v_old/strings.json
x ./s2v_old/._key2row
x ./s2v_old/key2row


In [3]:
# load the sense2vec vectors using the comments from reddit
from sense2vec import Sense2Vec
s2v = Sense2Vec().from_disk('s2v_old')

In [7]:
# Steps to find distractors for "Donald Trump"
word = 'work'
# Convert the string to lower case letters
word = word.lower()
# Replace space with underscore(_)
word = word.replace(' ','_')
print(word)

work


In [8]:
# Find the distractors by feeding the word to sense2vec
sense = s2v.get_best_sense(word)
# Get the sense
print("Best Sense",sense)

Best Sense work|VERB


In [9]:
# Now find the most similar words for this word in this sense
most_similar = s2v.most_similar(sense,n=12)
print(most_similar)

[('working|VERB', 0.8359), ('worked|VERB', 0.8057), ('works|VERB', 0.8027), ('working|NOUN', 0.789), ('only_work|NOUN', 0.7675), ('manage|VERB', 0.7515), ('_work|NOUN', 0.75), ('work|NOUN', 0.7466), ('Working|VERB', 0.7341), ('work-|VERB', 0.7341), ('do|VERB', 0.7314), ('at_least_work|NOUN', 0.724)]


In [10]:
# Packaging this into a callable function
from collections import OrderedDict
def sense2Vec_getSimilarWords(word,s2v):
  # Declare the output and its type - arrays or list
  similarWords = []

  # String processing - converting to lower and introducing a underscore for blanks
  word = word.lower().replace(' ','_')

  # Get the best sense for the given word
  sense = s2v.get_best_sense(word)
  most_Similar = s2v.most_similar(sense,n=20)

  for each_word in most_Similar:
    similarWords.append(each_word[0].split("|")[0].replace("_"," "))

  return similarWords

In [11]:
sense2Vec_getSimilarWords("Natural Language Processing",s2v)

['machine learning',
 'computer vision',
 'deep learning',
 'data analysis',
 'neural nets',
 'relational databases',
 'algorithms',
 'neural networks',
 'data processing',
 'image recognition',
 'NLP',
 'big data',
 'Machine Learning',
 'NLP',
 'data science',
 'big data analysis',
 'information retrieval',
 'speech recognition',
 'Machine learning',
 'programming languages']

# Fasttext

Trying wordembedings from Fast Text  https://fasttext.cc/docs/en/support.html#building-fasttext-python-module

In [1]:
# !git clone https://github.com/facebookresearch/fastText.git
# !cd fastText
# !pip install fastText
import fasttext.util
#fasttext.util.download_model('it', if_exists='ignore')  # English
model = fasttext.load_model('cc.it.300.bin')

In [12]:
# !git clone https://github.com/facebookresearch/fastText.git
# !cd fastText
# !sudo pip install .
# or :
# !sudo python setup.py install

Cloning into 'fastText'...
'sudo' is not recognized as an internal or external command,
operable program or batch file.


In [13]:
# import fasttext.util
# fasttext.util.download_model('it', if_exists='ignore')  # English


In [3]:
model = fasttext.load_model('cc.it.300.bin')

In [8]:
model.get_nearest_neighbors('numero ordinale')

[(0.69635009765625, 'dinale'),
 (0.6637539267539978, 'tecnicoeconomica'),
 (0.6575295329093933, 'numeroe'),
 (0.6210983991622925, 'ordinal'),
 (0.617618203163147, 'Ordinale'),
 (0.6075699925422668, 'Numeroordinale'),
 (0.6015861630439758, 'Numerocardinale'),
 (0.5819660425186157, 'estetologo'),
 (0.5808798670768738, 'Sindacatore'),
 (0.5799951553344727, 'anticardinale')]

In [10]:
model.get_analogies("numero ordinale", "carinale", "millesimo")

[(0.4903110861778259, 'decimillesimo'),
 (0.46558308601379395, 'millesimi'),
 (0.430698961019516, 'duecentesimo'),
 (0.4299042224884033, 'ordinalità'),
 (0.4258556365966797, 'centesimo'),
 (0.421135812997818, 'decimo'),
 (0.4166191816329956, 'trecentesimo'),
 (0.40764376521110535, 'decimillesimi'),
 (0.4020084738731384, 'cinquantesimo'),
 (0.40015530586242676, 'milionesimo')]

# 4. Word Embeddings and Language Models

Steps in order to achive distractors from the combination of word embeddings and Laguage models:

1. Data Collection

2. Preprocessing

3. Word Embeddings

4. Similarity Calculation

5. Synonym Replacement

6. Lexical Substitution

7. Language Models

8. Evaluation

9. Iterative Improvement

10. Testing

## Insert distractors into the dataset

In [38]:
with open("hypernym_discovery-task26-train-data.jsonl", 'r') as file:
        count = 0  # Counter to track processed pairs
        for line in file:
            if count >= 5:  # Break the loop if 5 pairs have been processed
                break
            pair = json.loads(line)
            id = pair["id"]
            choices = pair["choices"]
            label = pair["label"]
            print(f"\nTrue values: {choices[label]}")
            print(f"Predicted values: {model.get_nearest_neighbors(choices[label])}")
            count += 1
        


True values: grado
Predicted values: [(0.6741050481796265, 'gradodi'), (0.6480050683021545, 'capace'), (0.641866147518158, 'capaci'), (0.5582722425460815, 'grado.'), (0.554225504398346, 'ingrado'), (0.5511019825935364, 'poter'), (0.5481618642807007, 'grado.Il'), (0.5384020209312439, 'grado-'), (0.5263723134994507, 'livello'), (0.5168741345405579, 'saper')]

True values: numero ordinale
Predicted values: [(0.69635009765625, 'dinale'), (0.6637539267539978, 'tecnicoeconomica'), (0.6575295329093933, 'numeroe'), (0.6210983991622925, 'ordinal'), (0.617618203163147, 'Ordinale'), (0.6075699925422668, 'Numeroordinale'), (0.6015861630439758, 'Numerocardinale'), (0.5819660425186157, 'estetologo'), (0.5808798670768738, 'Sindacatore'), (0.5799951553344727, 'anticardinale')]

True values: frazione
Predicted values: [(0.7035280466079712, 'Frazione'), (0.6789522767066956, 'frazioni'), (0.6484289169311523, 'frazioncina'), (0.616841733455658, 'sottofrazione'), (0.6167787313461304, 'borgata'), (0.598190

# 4. Load model for evaluation

In [36]:
# Device agnostic code
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
# Configuration
runtimeFlag = device
cache_dir = None
scaling_factor = 1.0

In [ ]:
!pip install -q -U transformers peft accelerate optimum

!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.9/409.9 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
!pip install -q -U pdfminer.six # could maybe add pre-built wheels to speed this up.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 56.7 MB/s eta 0:00:00


In [ ]:
# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
# Load Model
import transformers
import torch
import json
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_id = "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"
tokenizer_llama = AutoTokenizer.from_pretrained(model_id)
model_llama = AutoModelForSequenceClassification.from_pretrained(model_id).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

## Perfomance Evaluation 

In [ ]:
import json
import torch
import torch.nn.functional as F

with open('train_dataset.json', 'r') as file:
    entities = json.load(file)[:6]
    for entity in entities:
      print(entity)
      id = entity['id']
      text = entity['text']
      choices = entity['choices']
      label = entity['label']

      prompt_formatted = f"Il termine '{text}' può essere iperonimo di: {choices}"
      input = tokenizer_llama(prompt_formatted, return_tensors="pt").to(device)
      output = model_llama(**input)

      probabilities = F.softmax(output.logits, dim=-1)
      probabilities = probabilities.squeeze().tolist()

      # # Find the index of the choice with the highest probability
      # for choice, prob in zip(choices, probabilities):
      #       print(f"Choice: {choice}, Probability: {prob}\n")

      logits = output.logits[0].detach().cpu().numpy()

      probabilities = torch.softmax(torch.tensor(logits), -1).detach().cpu().numpy()
      print(f"Probabilities: {probabilities}")
      prediction_index = probabilities.argmax()
      prediction_label = choices[prediction_index]
      prediction_confidence = probabilities[prediction_index] * 100
      prediction = {"label": prediction_index, "confidence": round(prediction_confidence, 1)}

      is_correct = prediction_index == label

      print("Prompt:", prompt_formatted)
      print("Prediction:", prediction)
      print("Actual Label:", label)
      print("Correct Prediction:", is_correct)
      print()

{'id': 0, 'text': 'sesto', 'choices': ['grado', 'casa reale', 'fulminazione', 'confetto'], 'label': 0}
Probabilities: [5.749090e-04 9.990707e-01 3.544809e-04]
Prompt: Il termine 'sesto' può essere iperonimo di: ['grado', 'casa reale', 'fulminazione', 'confetto']
Prediction: {'label': 1, 'confidence': 99.9}
Actual Label: 0
Correct Prediction: False

{'id': 0, 'text': 'sesto', 'choices': ['numero ordinale', 'innovazione', 'specialmente', 'casalmaggiore'], 'label': 0}
Probabilities: [4.6423267e-04 9.9934262e-01 1.9315351e-04]
Prompt: Il termine 'sesto' può essere iperonimo di: ['numero ordinale', 'innovazione', 'specialmente', 'casalmaggiore']
Prediction: {'label': 1, 'confidence': 99.9}
Actual Label: 0
Correct Prediction: False

{'id': 0, 'text': 'sesto', 'choices': ['frascheggiare', 'frazione', 'particella', 'avvinghiare'], 'label': 1}
Probabilities: [8.4733742e-04 9.9877518e-01 3.7740712e-04]
Prompt: Il termine 'sesto' può essere iperonimo di: ['frascheggiare', 'frazione', 'particella'

# 5. Roberta

In [ ]:
import json
import torch
from torch.distributions import Categorical
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("LIAMF-USP/roberta-large-finetuned-race")
model = AutoModelForSequenceClassification.from_pretrained("LIAMF-USP/roberta-large-finetuned-race")

tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at LIAMF-USP/roberta-large-finetuned-race and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
